<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<font color="#138D75">**WEkEO Training Service**</font> <br>
**Copyright:** 2023 EUMETSAT <br>
**License:** MIT <br>
**Authors:** Anna-Lena Erdmann (EUMETSAT)

<div class="alert alert-block alert-success">
<h3>xcube Viewer: Create and Visualize a Data Cube using WEkEO and the xcube Viewer</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
  - **<a href="https://my.wekeo.eu/user-registration" target="_blank">A WEkEO account</a>** if you are using or plan to use WEkEO.
  - access and execution of this notebook inside the **<a href="https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwi3tvDwzdqFAxWCh_0HHSo1DwEQFnoECA8QAQ&url=https%3A%2F%2Fjupyterhub.prod.wekeo2.eu%2F&usg=AOvVaw1qgpIG2_El9SA6di0OfwQ6&opi=89978449" target="_blank"> WEkEO JupyterHub</a>**
  - some knowledge about the data access through the **<a href="https://github.com/wekeo/wekeo4data/blob/main/wekeo-earthkit/01_WEkEOEarthkit_Introduction.ipynb" target="_blank">Earthkit WEkEO Plugins</a>**

</div>
<hr>

# 1 Create and Visualize a Data Cube using WEkEO and the xcube Viewer

### Data used

| Product Description  | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-------------:|:-----------------:|
| ERA5 hourly data on single levels from 1940 to present  | EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AECMWF%3ADAT%3AREANALYSIS_ERA5_SINGLE_LEVELS" target="_blank">link</a> | 

### Learning outcomes

At the end of this notebook you will know;

* how to create xcube-compatible data cubes from WEkEO data 
* how to correctly write a config file for xcube Viewer
* how to visualize and interact with data in the xcube viewer

### Outline

The <a href="https://xcube.readthedocs.io/en/latest/index.html" target="_blank">xcube</a> software package has been developed to generate, manipulate, analyse, and publish data cubes from Earth Observation data. This notebook shows how xcube, especially the xcube viewer can be used with EO data coming from WEkEO.

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Import](#section0)
 2. [Download Datasets with the Earthkit Plugin](#section1)
 3. [Creating a Datacube with xArray](#section2)
 4. [Creating a config file](#section3)
 5. [Viewing the Data with xcube Viewer](#section4)

<hr>

<div class="alert alert-info" role="alert">

## 1. <a id='section0'></a>Import
[Back to top](#TOC_TOP)
    
</div>

In [1]:

import os                                       # a library that allows us access to basic operating system commands like making directories    
import earthkit                                 # a library for seamless data access and conversion to xarray
from earthkit.data import settings, cache       # modules helpful for configuring the earthkit package
import xarray as xr                             # a library that supports the use of multi-dimensional arrays in Python
from xcube.webapi.viewer import Viewer          # a library to visualize datacubes inside the Jupyter Notebook 


Set the xcube Viewer url to show the xcube Viewer inside the notebook. 

**Note: Replace the `<your_username>` with your WEkEO username.**

In [2]:
os.environ["XCUBE_JUPYTER_LAB_URL"] = "https://jupyterhub.prod.wekeo2.eu/user/<your_username>/"

<div class="alert alert-info" role="alert">

## 2. <a id='section1'></a>Download Datasets using the Earthkit Plugin
[Back to top](#TOC_TOP)
    
</div>

In this notebook we will work with the **ERA5 Air temperature reanalysis**. You can get more information on this dataset and explore it <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AECMWF%3ADAT%3AREANALYSIS_ERA5_SINGLE_LEVELS" target="_blank">here</a>. 

The API request is derived from the WEkEO Viewer. 

We will download and transform the data to xarray using the CliMetLab WEkEO Plugin.

When using the climetlab functions the first time, you will have to enter WEkEO username and password before the data download.  

In [2]:
s = {"cache-policy": "user",
     "user-cache-directory": "./cache"}
settings.set(s)
cache.directory()

'./cache'

In [3]:
era5_request = earthkit.data.from_source("wekeo-source", {
  "dataset_id": "EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS",
  "product_type": ["reanalysis"],
  "variable": ["2m_temperature"],
  "year": ["2019"],
  "month": ["01"],
  "day": ["01"], 
  "time": ["00:00","01:00","02:00","03:00","04:00","05:00",
            "06:00","07:00","08:00","09:00","10:00","11:00",
            "12:00","13:00","14:00","15:00","16:00","17:00",
            "18:00","19:00","20:00","21:00","22:00","23:00"],
  "data_format": "netcdf",
  "download_format": "zip",
  "itemsPerPage": 200,
  "startIndex": 0
})

In [4]:
# download dataset and convert it to xarray

era5_ds = era5_request.to_xarray()

In [5]:
era5_ds

<xarray.Dataset> Size: 100MB
Dimensions:     (valid_time: 24, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 192B 2019-01-01 ... 2019-01-01T23...
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver      (valid_time) <U4 384B dask.array<chunksize=(24,), meta=np.ndarray>
Data variables:
    t2m         (valid_time, latitude, longitude) float32 100MB dask.array<chunksize=(12, 361, 720), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-10T09:27 GRIB to CDM+CF via cfgrib-0.9.1...

<div class="alert alert-info" role="alert">

## 3. <a id='section2'></a>Creating a Datacube with xArray
[Back to top](#TOC_TOP)
    
</div>

From the xarray overview we see, that the longitude is in the range of `[0, 360]`. For compatibility reasons we convert it to the range of `[-180, 180]` and remane the time variable to `time`. 

In [6]:
era5_ds.coords['longitude'] = (era5_ds.coords['longitude'] + 180) % 360 - 180
era5_ds = era5_ds.rename({"valid_time": "time"})
era5_ds = era5_ds.drop_vars(["number", "expver"])
era5_ds_reshaped = era5_ds.sortby(era5_ds.longitude)



Subset the global dataset to our area of interest os Spain:

In [7]:
spain_era5 = era5_ds_reshaped.sel( latitude=slice(44.1, 35.6), longitude = slice(-10, 4))[['t2m']]
spain_era5.attrs["title"] = "Air Temp Spain 2023-01-01"

In [8]:
spain_era5

<xarray.Dataset> Size: 187kB
Dimensions:    (time: 24, latitude: 34, longitude: 57)
Coordinates:
  * time       (time) datetime64[ns] 192B 2019-01-01 ... 2019-01-01T23:00:00
  * latitude   (latitude) float64 272B 44.0 43.75 43.5 ... 36.25 36.0 35.75
  * longitude  (longitude) float64 456B -10.0 -9.75 -9.5 -9.25 ... 3.5 3.75 4.0
Data variables:
    t2m        (time, latitude, longitude) float32 186kB dask.array<chunksize=(12, 34, 40), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-02-10T09:27 GRIB to CDM+CF via cfgrib-0.9.1...
    title:                   Air Temp Spain 2023-01-01

<hr>
<div class="alert alert-info" role="alert">

## 4. <a id='section3'></a>Creating a config file
[Back to top](#TOC_TOP)

</div>


To add data cubes to the xcube viewer, they have to be defined in a config file.

Two main parts of the config file is the ``Datasets`` description and the ``Styles`` description. As our datacube is already in our environment, we don't have to explicitly define the ``Datasets`` parameters it in the config file. So we are only setting the ``Styles`` parameters. 

The ``Style`` description contains:
* **Identifier** *unique name for the style*
* **ColorMappings** 
    * **VariableName** *must be identical to the variable name in the cube*
        * **ColorBar** *name of colorbar as in matplotlib*
        * **ValueRange** *range of the dataset*

In [9]:
viewer = Viewer(server_config={
    "Styles": [
        {
            "Identifier": "temperature",
            "ColorMappings": {
                "t2m": {
                    "ValueRange": [250.0, 300.0],
                    "ColorBar": "inferno"
                },
            }
        }
    ]
})

Next, we add our dataset to the Viewer

In [10]:
viewer.add_dataset(spain_era5, style="temperature")

'5d44a471-d768-4800-96a9-d22f8de1ac9d'

<hr>
<div class="alert alert-info" role="alert">

## 5. <a id='section4'></a>Viewing the Data with xcube Viewer
[Back to top](#TOC_TOP)

</div>


The `viewer.show()` function shows an instance of the xcube Viewer inside your norebook. 

You can now explore the datacubes by <font color="#138D75"> browse through time, create a time series at one point, compare different locations ...</font>

In [11]:
viewer.show()

404 GET /viewer/config/config.json (127.0.0.1): xcube viewer has not been been configured
404 GET /viewer/config/config.json (127.0.0.1) 4.00ms
c:\Users\erdmann\AppData\Local\Programs\Python\Python311\Lib\site-packages\xcube\core\schema.py:464: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  dim_name: cube_chunks.get(dim_name, cube.dims[dim_name])
c:\Users\erdmann\AppData\Local\Programs\Python\Python311\Lib\site-packages\xcube\core\timecoord.py:248: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  return getattr(timestamp, round_fn)(freq).isoformat() + 'Z'
Uncaught exception GET /datasets?details=1 (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:8002', method='GET', uri='/datasets?details=1', version='HTTP/1.1', remote_ip='1

<img src='./img/01_xcubeviewer_show.png' align='left' width='80%'/>